# Grundlagen der KI - Gruppenprojekte


Für die Projekte stellen wir eine MongoDB Datenbank, die wir aktiv in der Forschung nutzen, zur Verfügung. Die Datenbank beinhaltet Informationen über Versionskontrollsysteme, Issuetracker, und Mailinglisten von Open Source Projekten. [Die Dokumentation der Datenbank finden Sie Online](https://smartshark2.informatik.uni-goettingen.de/documentation/).

## Die Aufgabe

Ihre Aufgabe ist es, einen automatisierten Ansatz zu entwickeln, welcher vorhersagt, ob ein Issue innerhalb von 6 Monaten behoben wird oder nicht. Darüber hinaus, sollte das Modell (oder auch ein zweites Modell) in der Lage sein, die erwartete Zeit für die Behebung und damit Schließung des Issues vorherzusagen.
Sie können dazu beliebige Informationen aus der Datenbank nutzen, wie z.B. Commits, andere Issues, Mails und Änderungen in den Dateien.
Sie müssen diese Problemstellung als Analyseproblem fassen um dann Modelle zur Lösung aufstellen zu können. Hierzu müssen Sie geeignete Analysemethoden auswählen und anschließend die Qualität der Ergebnisse evaluieren. 

## Präsentationen

**Alle Gruppen müssen ihre Ergebnisse in der letzten Vorlesung am 09.02. um 16:15 Uhr vorstellen.** Hierzu muss jede Gruppe eine kurze Präsentation halten. In dieser Präsentation sollten Sie kurz beschreiben wie Sie vorgegangen sind, wie Sie Features erstellt haben, welche Modelle Sie benutzt haben und was ihre wichtigsten Ergebnisse sind. Die Zeit für die Präsentation beträgt 9 (1) Minuten.

Im Anschluss an die Präsentation stimmen die Anwesenden über das beste Projekt ab. Jede Gruppe stimmt für das beste Projekt (3 Punkte), das zweitbeste (2 Punkte) und das drittbeste (1 Punkt). Das Projekt mit den meisten Punkten gewinnt einen Preis.

Die Präsentation wird in Präsenz stattfinden, wenn die aktuellen Regeln es zulassen (Prof. Herbold wird dabei per Video zugeschaltet sein; Raum wird noch bekannt gegeben). Da wir nur begrenzt Zeit haben, müssen alle Präsentationen vorab geschickt werden. Wir kombinieren diese dann in einer PDF um einen einfachen Wechsel zwischen den Präsentation der Gruppen zu erreichen. Die Mitglieder der Gruppe werden dann in der Reihenfolge des Gruppennames gebeten ihre Ergebnisse vorzustellen, d.h. wir beginnen mit Gruppe 1 und enden mit Gruppe 6.


## Abgabe der Präsentationen

Damit die Präsentation reibungslos ablaufen, müssen Sie ihre Präsentation im Vorfeld einsenden. **Schicken Sie Ihre Präsentationen bitte bis spätestens zum 08.02.23, 16 Uhr an Clemens Dierks via Email (clemens.dierks@tu-clausthal.de)**. Die Präsentation muss als PDF vorliegen.


## Mindestkriterien

Die folgenden Kriterien muss ein Projekt mindestens erfüllen, damit die Gruppenmitglieder an der Prüfung teilnehmen können.
- Es muss ein Modell für die automatisierte Vorhersage, ob ein Issue innerhalb von 6 Monaten gelöst wird, vorhanden sein.
- Es muss ein Modell für die automatisierte Vorhersage der estimated-time-to-fix von Issues vorhanden sein.
- Die Qualität des Modells muss bewertet werden.
- Sie müssen eine Empfehlung geben, ob und ggf. unter welchen Umständen, das Modell eingesetzt werden kann basierend auf der Evaluation der Qualität des Modells.
- Aufgrund Ihrer Empfehlung können Sie noch Verbesserungsmöglichkeiten vorstellen.
- Die Ergebnisse müssen in der Vorlesung am 09.02.23 vorgestellt werden.

## Datenbankzugriff

Unterhalb finden Sie ein Beispiel, wie Sie auf die Datenbank zugreifen. Die Datenbank befindet sich hinter einer Firewall und kann nur aus dem Netz der TU Clausthal erreicht werden. Wenn Sie Probleme haben die Datenbank zu erreichen, können Sie von jedem beliebigen Netzwerk aus in das Netz der TU Clausthal gelangen, in dem Sie sich mit dem [VPN](https://doku.tu-clausthal.de/doku.php?id=netzwerk_und_internet:vpn:start) verbinden.

**WARNUNG:
Wir benutzen die Datenbank aktiv in der Forschung. Es kann daher sein, das teilweise eine sehr hohe Last auf der Datenbank ist. Die Datenbank ist sehr groß und beinhaltet derzeit 4.8 Terabyte an Daten. Sie sollten daher nicht einfach starten Anfragen zu schreiben, sondern sich genau überlegen auf welche Informationen Sie zugreifen wollen. Wenn Sie zum Beispiel eine Anfrage stellen, in der Sie die komplette ```commit``` Collection herunterladen möchten, müssen Sie hierfür 158 Gigabyte an Daten runterladen. **

Sie können die Bibliothek [pycoSHARK](https://github.com/smartshark/pycoshark) zum Zugriff auf die Datenbank nutzen. Der pycoSHARK stellt einen ORM-Layer basierend auf der Bibliothek ```mongoengine``` zur Verfügung. Hierdurch können Sie die Datenbanktabellen wie Objekte behandeln. Alternativ können Sie die native MongoDB API der ```pymongo``` Bibliothek benutzen. 

Der Quelltext unten beinhaltet Ihre Zugangsdaten in zeigt wie man mit dem pycoSHARK auf die Datenbank zugreift.

Interesting features: No. of commits of the issue creator,age of the branch 

In [2]:
# Installation des pycoSHARKs
import sys
# !{sys.executable} -m pip install pycoshark

In [3]:
from mongoengine import connect, disconnect
from pycoshark.mongomodels import File, FileAction, Commit, Project, VCSSystem, IssueSystem,Hunk, Refactoring, IssueSystem, Issue, IssueComment, MailingList, Message
from pycoshark.utils import create_mongodb_uri_string
import pandas as pd
import pandas_profiling as pp #best to be installed with anaconda since pip sometimes makes problems with pandas_profiling
# Database credentials
user = 'grundlagenki2021'
password = '38BTGx71d'
host = '134.76.81.151'
port = '27017'
authentication_db = 'smartshark'
database = "smartshark"
ssl_enabled = None

# Establish connection
uri = create_mongodb_uri_string(user, password, host, port, authentication_db, ssl_enabled)
connect(database, host=uri)

# Fetch project id and issue tracking system system id for the 'commons-math' project
# The only() decides the data that is actually retrieved from the MongoDB. 
# get() only works, when you query results in exactly one document, otherwise you have to use a loop to access the data
# an example with a loop is at the bottom of this example
project = Project.objects(name='commons-math').only('id').get()
issue_system = IssueSystem.objects(project_id=project.id).only('id','url').get()
print('url of issue system of the project: %s' % issue_system.url)

url of issue system of the project: https://issues.apache.org/jira/rest/api/2/search?jql=project=MATH


Copied from https://github.com/smartshark/usage-examples/blob/main/Example-Notebook.ipynb:

In [47]:
# We first need the the project from the database
project = Project.objects(name='commons-math').get()

# We now select the issue tracking system of the project
# Please note that some projects have multiple issue trackers
# In this case get() would fail and you would need to loop over them
issue_tracker = IssueSystem.objects(project_id=project.id).get()

print('Issue Tracker:', issue_tracker.url)
print('String zum Nachschlagen der korrekten external IDs:', issue_tracker.url.rsplit('=', 1)[1])# 

# we can now work with the issues
num_issues = Issue.objects(issue_system_id=issue_tracker.id).count()

print('Number of issues:', num_issues)

count_comments = 0
count_referenced_by_commits = 0
count_bugs_dev_label = 0
count_bugs_validated = 0

for issue in Issue.objects(issue_system_id=issue_tracker.id):
    count_comments += IssueComment.objects(issue_id=issue.id).count()
    if issue.issue_type is not None and issue.issue_type.lower()=='bug':
        count_bugs_dev_label += 1
    if issue.issue_type_verified is not None and issue.issue_type_verified.lower()=='bug':
        count_bugs_validated += 1
    if Commit.objects(linked_issue_ids=issue.id).count()>0:
        count_referenced_by_commits += 1
        
print('Number of comments in discussions:', count_comments)
print('Number of issues referenced by commits:', count_referenced_by_commits)
print('Number of issues labeled as bugs by developers:', count_bugs_dev_label)
print('Number of issues labeled validated as bug by researchers:', count_bugs_validated)


Issue Tracker: https://issues.apache.org/jira/rest/api/2/search?jql=project=MATH
Issue Tracker: MATH
Number of issues: 1455
Number of comments in discussions: 7640
Number of issues referenced by commits: 938
Number of issues labeled as bugs by developers: 673
Number of issues labeled validated as bug by researchers: 244


In [6]:
for items in Project.objects:
    print (items.name)

commons-math
oisafe
ant-ivy
junit
kafka
zeppelin
mahout
struts
opennlp
pig
calcite
cayenne
falcon
kylin
nutch
parquet-mr
tika
wss4j
archiva
deltaspike
jspwiki
xerces2-j
tez
systemml
storm
lens
accumulo
commons-lang
james
pdfbox
flume
nifi
knox
derby
commons-collections
commons-beanutils
commons-codec
commons-compress
commons-configuration
commons-digester
commons-imaging
commons-io
commons-jcs
commons-jexl
commons-rdf
commons-scxml
commons-validator
commons-vfs
giraph
manifoldcf
ranger
eagle
commons-bcel
commons-dbcp
zookeeper
phoenix
gora
santuario-java
helix
jena
httpcomponents-client
commons-net
httpcomponents-core
streams
samza
roller
mina-sshd
bigtop
jackrabbit
tomcat
rhino
oozie
avro
activemq
fop
maven
airavata
openjpa
AutoUpdater.NET
directory-fortress-core
eShopOnWeb
EquinoxProject
BlazorWithIdentity
ruler
quick-picture-viewer
Mynt
SuperSafeBank
Hunt
Windows-appsample-networkhelper
Windows-appsample-customers-orders-database
Miniblog.Core
coolstore-microservices
clean-architect

In [49]:
# We first need the the project from the database
project = Project.objects(name='calcite').get()

# We now select the issue tracking system of the project
# Please note that some projects have multiple issue trackers
# In this case get() would fail and you would need to loop over them
issue_tracker = IssueSystem.objects(project_id=project.id).get()
df=pd.read_json(Issue.objects(issue_system_id=issue_tracker.id).to_json())


In [50]:
df.head()

,_id,external_id,issue_system_id,title,created_at,updated_at,creator_id,reporter_id,issue_type,priority,...,resolution,fix_versions,assignee_id,issue_links,desc,environment,issue_type_manual,issue_type_verified,original_time_estimate,parent_issue_id
0,{'$oid': '5b3b320226b42f23be17da1a'},CALCITE-324,{'$oid': '5b3b301d26b42f23be17d4f0'},When doing view expansion using frameworks int...,{'$date': 1404322454585},{'$date': 1408993815874},{'$oid': '5b3b320030a71b06bc6c8ea1'},{'$oid': '5b3b320030a71b06bc6c8ea1'},Improvement,Major,...,Fixed,[0.9.0-incubating],{'$oid': '5b3b320030a71b06bc6c8ea1'},[],NaN,NaN,NaN,NaN,NaN,NaN
1,{'$oid': '5b3b320426b42f23be17da22'},CALCITE-323,{'$oid': '5b3b301d26b42f23be17d4f0'},"When late function binding, Optiq will have is...",{'$date': 1404322288790},{'$date': 1408993815375},{'$oid': '5b3b320030a71b06bc6c8ea1'},{'$oid': '5b3b320030a71b06bc6c8ea1'},Bug,Major,...,Fixed,[0.9.0-incubating],{'$oid': '5b3b320030a71b06bc6c8ea1'},[],NaN,NaN,NaN,NaN,NaN,NaN
2,{'$oid': '5b3b320626b42f23be17da2a'},CALCITE-327,{'$oid': '5b3b301d26b42f23be17d4f0'},Rules should use base class to find rule match...,{'$date': 1404336659511},{'$date': 1408993815047},{'$oid': '5b3b320630a71b06bc6cc667'},{'$oid': '5b3b320630a71b06bc6cc667'},Bug,Major,...,Fixed,[0.9.0-incubating],{'$oid': '5b35273530a71b06bc5391ba'},[],Rules should use base class to find rule match...,NaN,NaN,NaN,NaN,NaN
3,{'$oid': '5b3b320726b42f23be17da32'},CALCITE-300,{'$oid': '5b3b301d26b42f23be17d4f0'},Support multiple parameters in count(distinct ...,{'$date': 1403082320047},{'$date': 1408993815705},{'$oid': '5b3b302130a71b06bc5042e9'},{'$oid': '5b3b302130a71b06bc5042e9'},Improvement,Major,...,Fixed,[0.9.0-incubating],NaN,[],Currently for sql like -- select count(distinc...,NaN,NaN,NaN,NaN,NaN
4,{'$oid': '5b3b320826b42f23be17da40'},CALCITE-377,{'$oid': '5b3b301d26b42f23be17d4f0'},"UnregisteredDriver should catch, log and re-th...",{'$date': 1408339241368},{'$date': 1408993816743},{'$oid': '5b35273530a71b06bc5391ba'},{'$oid': '5b35273530a71b06bc5391ba'},Bug,Major,...,Fixed,[0.9.0-incubating],{'$oid': '5b35273530a71b06bc5391ba'},[],"UnregisteredDriver should catch, log and re-th...",NaN,NaN,NaN,NaN,NaN


In [52]:

df['created_at']=pd.json_normalize(df['created_at'])
df['updated_at']=pd.json_normalize(df['updated_at'])

df['timetofix']=df.updated_at-df.created_at


ValueError: Columns must be same length as key

In [53]:
df.head()

,_id,external_id,issue_system_id,title,created_at,updated_at,creator_id,reporter_id,issue_type,priority,...,fix_versions,assignee_id,issue_links,desc,environment,issue_type_manual,issue_type_verified,original_time_estimate,parent_issue_id,timetofix
0,{'$oid': '5b3b320226b42f23be17da1a'},CALCITE-324,{'$oid': '5b3b301d26b42f23be17d4f0'},When doing view expansion using frameworks int...,1.404322e+12,1.408994e+12,{'$oid': '5b3b320030a71b06bc6c8ea1'},{'$oid': '5b3b320030a71b06bc6c8ea1'},Improvement,Major,...,[0.9.0-incubating],{'$oid': '5b3b320030a71b06bc6c8ea1'},[],NaN,NaN,NaN,NaN,NaN,NaN,4.671361e+09
1,{'$oid': '5b3b320426b42f23be17da22'},CALCITE-323,{'$oid': '5b3b301d26b42f23be17d4f0'},"When late function binding, Optiq will have is...",1.404322e+12,1.408994e+12,{'$oid': '5b3b320030a71b06bc6c8ea1'},{'$oid': '5b3b320030a71b06bc6c8ea1'},Bug,Major,...,[0.9.0-incubating],{'$oid': '5b3b320030a71b06bc6c8ea1'},[],NaN,NaN,NaN,NaN,NaN,NaN,4.671527e+09
2,{'$oid': '5b3b320626b42f23be17da2a'},CALCITE-327,{'$oid': '5b3b301d26b42f23be17d4f0'},Rules should use base class to find rule match...,1.404337e+12,1.408994e+12,{'$oid': '5b3b320630a71b06bc6cc667'},{'$oid': '5b3b320630a71b06bc6cc667'},Bug,Major,...,[0.9.0-incubating],{'$oid': '5b35273530a71b06bc5391ba'},[],Rules should use base class to find rule match...,NaN,NaN,NaN,NaN,NaN,4.657156e+09
3,{'$oid': '5b3b320726b42f23be17da32'},CALCITE-300,{'$oid': '5b3b301d26b42f23be17d4f0'},Support multiple parameters in count(distinct ...,1.403082e+12,1.408994e+12,{'$oid': '5b3b302130a71b06bc5042e9'},{'$oid': '5b3b302130a71b06bc5042e9'},Improvement,Major,...,[0.9.0-incubating],NaN,[],Currently for sql like -- select count(distinc...,NaN,NaN,NaN,NaN,NaN,5.911496e+09
4,{'$oid': '5b3b320826b42f23be17da40'},CALCITE-377,{'$oid': '5b3b301d26b42f23be17d4f0'},"UnregisteredDriver should catch, log and re-th...",1.408339e+12,1.408994e+12,{'$oid': '5b35273530a71b06bc5391ba'},{'$oid': '5b35273530a71b06bc5391ba'},Bug,Major,...,[0.9.0-incubating],{'$oid': '5b35273530a71b06bc5391ba'},[],"UnregisteredDriver should catch, log and re-th...",NaN,NaN,NaN,NaN,NaN,6.545754e+08


In [55]:
df=df[['external_id', 'components', 'labels', 'creator_id', 'issue_type', 'priority', 'status' , 'timetofix']]

In [63]:
extid=issue_tracker.url.rsplit('=', 1)[1] # obtain the correct external_id for each repository to sort out invalid issues or logs
df=df[df.external_id.str.startswith(extid)]
#df[~df.external_id.str.startswith(extid)]# show removed rows

,external_id,components,labels,creator_id,issue_type,priority,status,timetofix


EDA lib taken from: https://pandas-profiling.ydata.ai/ 

In [1]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [62]:
pp.ProfileReport(df, title="Pandas Profiling Report")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

sort out not matching external ids:

,external_id,components,labels,creator_id,issue_type,priority,status,timetofix
0,CALCITE-324,[],[],{'$oid': '5b3b320030a71b06bc6c8ea1'},Improvement,Major,Closed,4.671361e+09
1,CALCITE-323,[],[],{'$oid': '5b3b320030a71b06bc6c8ea1'},Bug,Major,Closed,4.671527e+09
2,CALCITE-327,[],[],{'$oid': '5b3b320630a71b06bc6cc667'},Bug,Major,Closed,4.657156e+09
3,CALCITE-300,[],[github-import],{'$oid': '5b3b302130a71b06bc5042e9'},Improvement,Major,Closed,5.911496e+09
4,CALCITE-377,[],[],{'$oid': '5b35273530a71b06bc5391ba'},Bug,Major,Closed,6.545754e+08
5,CALCITE-354,NaN,NaN,{'$oid': '5b35273530a71b06bc5391ba'},Bug,Major,Closed,2.763032e+09
6,CALCITE-351,[],[],{'$oid': '5b35273530a71b06bc5391ba'},New Feature,Major,Closed,2.845276e+09
8,CALCITE-345,[],[],{'$oid': '5b35273530a71b06bc5391ba'},Bug,Major,Closed,3.628964e+09
9,CALCITE-194,NaN,[github-import],{'$oid': '5b3b302130a71b06bc5042e9'},Bug,Major,In Progress,1.263360e+11
10,CALCITE-342,[],[],{'$oid': '5b3b320f30a71b06bc6d14c2'},Bug,Major,Closed,3.918538e+09
